In [1]:
import numpy as np
import polars as pl

from constants import *

In [10]:
df = pl.read_csv("datasets/wikidata5m_text.txt", has_header=True, encoding="utf8-lossy")

## Sentence transformers

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

In [11]:
sentences = df["description"].to_list()

In [ ]:
all_embeddings = []
step = 100_000

for cycle in range(0, len(sentences), step):
    embeddings = model.encode(sentences[cycle:cycle+step], show_progress_bar=True)
    all_embeddings.append(embeddings)

In [ ]:
embeddings = np.concatenate(all_embeddings)

In [ ]:
np.save(embeddings_file, embeddings)